#  IA para Redes de Suministro 

👤 **Autor:** John Leonardo Vargas Mesa  
🔗 [LinkedIn](https://www.linkedin.com/in/leonardovargas/) | [GitHub](https://github.com/LeStark)  

## 📂 Repositorio en GitHub  
- 📓 **Notebooks:** [Acceder aquí](https://github.com/LeStark/Cursos/tree/main/02%20-%20IA4SC)  
- 📑 **Data sets:** [Acceder aquí](https://github.com/LeStark/Cursos/tree/main/00%20-%20Data/02%20-%20SC)  
---

# 🧮 Red Neuronal que Aprende las 4 Operaciones Básicas
En este notebook vamos a entrenar una red neuronal para que aprenda a realizar:
- Suma
- Resta
- Multiplicación
- División



In [2]:
import pandas as pd
# Preprocesamiento de datos
from sklearn.preprocessing import LabelEncoder    # Codificación de variables categóricas
from sklearn.preprocessing import StandardScaler  # Estandarización de variables numéricas

# 1. Importar librerías necesarias
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
url = "https://raw.githubusercontent.com/LeStark/Cursos/refs/heads/main/00%20-%20Data/00%20-%20Basicos/dataset_operaciones.csv"
data = pd.read_csv(url)
data.head()


,x1,x2,operacion,resultado
0,38,72,+,110.0
1,12,64,-,-52.0
2,87,94,-,-7.0
3,19,92,*,1748.0
4,58,95,*,5510.0


In [4]:
etiquetador = LabelEncoder()
data["operacion_num"] = etiquetador.fit_transform(data["operacion"])
data.head()

,x1,x2,operacion,resultado,operacion_num
0,38,72,+,110.0,1
1,12,64,-,-52.0,2
2,87,94,-,-7.0,2
3,19,92,*,1748.0,0
4,58,95,*,5510.0,0


In [5]:
dict(zip(etiquetador.classes_, etiquetador.transform(etiquetador.classes_)))

{'*': 0, '+': 1, '-': 2, '/': 3}

In [6]:
data = data.drop(columns=["operacion"])
data.head()

,x1,x2,resultado,operacion_num
0,38,72,110.0,1
1,12,64,-52.0,2
2,87,94,-7.0,2
3,19,92,1748.0,0
4,58,95,5510.0,0


In [7]:
nombre_columnas_numericas = data.drop(columns=[ "operacion_num", "resultado"]).select_dtypes(include=['int64', 'float64']).columns.tolist()
escalador = StandardScaler()
columnas_numericas = escalador.fit_transform(data[nombre_columnas_numericas])
columnas_numericas = pd.DataFrame(columnas_numericas, columns=nombre_columnas_numericas)
columnas_numericas.head()

,x1,x2
0,-0.427790,0.731949
1,-1.327421,0.454461
2,1.267668,1.495043
3,-1.085213,1.425671
4,0.264234,1.529729


In [8]:
data_lista = pd.concat([columnas_numericas, data[["operacion_num", "resultado"]]], axis=1)
data_lista.head()

,x1,x2,operacion_num,resultado
0,-0.427790,0.731949,1,110.0
1,-1.327421,0.454461,2,-52.0
2,1.267668,1.495043,2,-7.0
3,-1.085213,1.425671,0,1748.0
4,0.264234,1.529729,0,5510.0


In [20]:


# Supongamos que ya transformaste 'operacion' en numérica o one-hot
X = data_lista.drop(columns=["resultado"])   # entradas
y = data_lista["resultado"]                  # salida

# Dividir en 80% train y 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

print("Tamaño train:", X_train.shape, "Tamaño test:", X_test.shape)


Tamaño train: (18000, 3) Tamaño test: (2000, 3)


In [22]:
model = keras.Sequential([
    layers.Dense(16, activation="relu", input_shape=(3,)),
    layers.Dense(32, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(32, activation="relu"),
    layers.Dense(1, activation="linear")
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                │ (None, 16)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,809 (14.88 KB)

 Trainable params: 3,809 (14.88 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1307134.6250 - mae: 523.0345 - val_loss: 89918.1094 - val_mae: 148.6854
Epoch 2/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 31273.8008 - mae: 96.8747 - val_loss: 12061.2979 - val_mae: 73.3890
Epoch 3/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 9859.4502 - mae: 68.0651 - val_loss: 8475.4209 - val_mae: 64.0694
Epoch 4/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 7433.4316 - mae: 61.4404 - val_loss: 6742.8062 - val_mae: 59.9424
Epoch 5/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 5742.9277 - mae: 56.2556 - val_loss: 5121.4517 - val_mae: 54.2299
Epoch 6/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 4589.0107 - mae: 51.6485 - val_loss: 4042.1692 - val_mae: 50.1449
Epoch 7/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3806.9639 - mae: 47.4538 - val_loss: 3942.0620 - val_mae: 47.4575
Epoch 8/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3266.8569 - mae: 43.5525 - val_loss: 2916.

In [24]:
# Ejemplo de entrada
a = 5
b = 10
op = "+"  # puede ser +, -, * o /

# --- Escalar números (formato 2D) ---
a_scaled, b_scaled = escalador.transform([[a, b]])[0]
op_encoded = etiquetador.transform([op])[0]

entrada = np.array([[a_scaled, b_scaled, op_encoded]]).reshape(1, 3)
print("Entrada normalizada:", entrada)

Entrada normalizada: [[-1.56962928 -1.41858821  1.        ]]


c:\Users\jlvar\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [25]:
# Predicción con el modelo
pred = model.predict(entrada)

print(f"\n📌 El modelo predice: {a} {op} {b} ≈ {pred[0][0]:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step

📌 El modelo predice: 5 + 10 ≈ 11.93
